In [14]:
# Load some of the packages
import numpy as np

# Generate three random sequences (not time series yet)
M, S = 3, 100

procs = np.random.randint(100, size=(M,S))

In [15]:
# Load the Data class
from pynats.data import Data

# Create an unnamed dataset with 3 processes and 1000 observations (1 replication)
#   - dim_order specifies processes is the first dimension and samples/observations are the second
#   - normalise z-scores the data
data = Data(procs, dim_order='ps', normalise=True)

Added dataset "None" with properties: 3 processes, 100 observations, 1 replications


In [16]:
# Load the Calculator class
from pynats.calculator import Calculator

# Let's create an unnamed calculator
calc = Calculator(dataset=data)

Loading configuration file: /home/oliver/Dropbox/Workspace/code/research/pynats/pynats/config.yaml
*** Importing module .correlation
[0] Adding measure .correlation.pearsonr(x,y,{'cov_estimator': 'empirical'})...
Succesfully initialised with identifier "pearsonr_empirical"
[1] Adding measure .correlation.pearsonr(x,y,{'cov_estimator': 'empirical', 'squared': True})...
Succesfully initialised with identifier "pearsonr-sq_empirical"
[2] Adding measure .correlation.pearsonr(x,y,{'cov_estimator': 'ledoit_wolf'})...
Succesfully initialised with identifier "pearsonr_ledoit_wolf"
[3] Adding measure .correlation.pearsonr(x,y,{'cov_estimator': 'shrunk'})...
Succesfully initialised with identifier "pearsonr_shrunk"
[4] Adding measure .correlation.pearsonr(x,y,{'cov_estimator': 'oas'})...
Succesfully initialised with identifier "pearsonr_oas"
[5] Adding measure .correlation.pcor(x,y,{'cov_estimator': 'empirical'})...
Succesfully initialised with identifier "pcorr_empirical"
[6] Adding measure .co

In [17]:
calc.compute()

Processing [None: te_gaussian_k-1_kt-1_l-1_lt-1]: 100%|██████████| 80/80 [00:04<00:00, 16.07it/s]


In [19]:
# Should be an adjacency matrix of features x processes x processes
calc.adjacency.shape

(80, 3, 3)